In [1]:
import os
import pathlib
import re
import sys

import findspark
from operator import add
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext, Row, SparkSession
from pyspark.sql.functions import regexp_replace, col, trim, lower, desc

prj_home = pathlib.Path().resolve().parent.parent
spark_home = os.path.join(prj_home / 'spark-3.5.0-bin-hadoop3')
findspark.init(spark_home)

sc = SparkContext(master='', appName='PySparkSqlContext') # machine that spark runs and the number of worker threads
ssc = StreamingContext(sparkContext=sc, batchDuration=20)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/20 10:32:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/audioworkstation/Documents/WORKSPACE/LEARNING/spark_streaming_using_x/.venv/lib/python3.10/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [2]:
def get_spark_session_interface(sparkConf):
    if 'sparkSessionSingletonInstance' not in globals():
        globals()['sparkSessionSingletonInstance'] = SparkSession.builder.config(conf=sparkConf).getOrCreate()
    return globals()['sparkSessionSingletonInstance']

In [3]:
host = '127.0.0.1'
port = '5555'

lines = ssc.socketTextStream(host, int(port))
seprated = lines.flatMap(lambda l: l.split(' '))


In [4]:
def process(time, rdd):
    print(f"-.-.-.-.-.-.-.-.-.- %s -.-.-.-..-.-.-..-.-.-" % str(time))
    if not rdd.isEmpty():
        session = get_spark_session_interface(rdd.context.getConf())
        rows_data = rdd.map(lambda word: Row(word_col=word))
        words_df = session.createDataFrame(rows_data)
        words_df.createOrReplaceTempView('words')

        word_count_df = session.sql('select word_col, count(*) from words group by word_col')
        word_count_df.show()
    
    else:
        print("No stream received!")

seprated.foreachRDD(process)

In [5]:
ssc.start()

In [ ]:
# ssc.stop(stopGraceFully=True, stopSparkContext=True)
ssc.awaitTerminationOrTimeout(timeout=100)

-.-.-.-.-.-.-.-.-.- 2023-11-20 10:32:40 -.-.-.-..-.-.-..-.-.-
No stream received!


23/11/20 10:32:56 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/11/20 10:32:56 WARN BlockManager: Block input-0-1700472775800 replicated to only 0 peer(s) instead of 1 peers


-.-.-.-.-.-.-.-.-.- 2023-11-20 10:33:00 -.-.-.-..-.-.-..-.-.-


23/11/20 10:33:00 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/11/20 10:33:00 WARN BlockManager: Block input-0-1700472780600 replicated to only 0 peer(s) instead of 1 peers


+--------+--------+
|word_col|count(1)|
+--------+--------+
|   doin?|       1|
|     you|       1|
|     How|       1|
|  Hello.|       1|
+--------+--------+



...

+--------+--------+
|word_col|count(1)|
+--------+--------+
|     was|       1|
|    your|       1|
|     How|       1|
|    day?|       1|
|     how|       1|
|    you?|       1|
|     are|       1|
+--------+--------+



23/11/20 10:33:33 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/11/20 10:33:33 WARN BlockManager: Block input-0-1700472813000 replicated to only 0 peer(s) instead of 1 peers
23/11/20 10:33:35 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/11/20 10:33:35 WARN BlockManager: Block input-0-1700472815000 replicated to only 0 peer(s) instead of 1 peers
23/11/20 10:33:39 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/11/20 10:33:39 WARN BlockManager: Block input-0-1700472818800 replicated to only 0 peer(s) instead of 1 peers


-.-.-.-.-.-.-.-.-.- 2023-11-20 10:33:40 -.-.-.-..-.-.-..-.-.-


+--------+--------+
|word_col|count(1)|
+--------+--------+
|     was|       1|
|   thing|       1|
|      ok|       2|
|   every|       1|
|       ?|       2|
|     ok?|       1|
+--------+--------+



-.-.-.-.-.-.-.-.-.- 2023-11-20 10:34:00 -.-.-.-..-.-.-..-.-.-
No stream received!
